In [1]:
import requests
import lxml.html as lh
import pandas as pd
import csv

In [2]:
url = 'http://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/'
uri_pages = '?page='
nbPages = 500
tags = ['//span[@class="stareval-note"]', '//div[@class="content-txt review-card-content"]']
cols = ['Note', 'Description']

In [20]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [21]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags) - 1):
        liste = liste.join(df_liste[j + 1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']

    return liste

In [22]:
getPage(url).head()

,Note,Description,key
0,"5,0",Après le chef d'oeuvre super-héroïque The Dark...,0
1,"5,0",C’est fou ce qu’on aime détester Christopher N...,1
2,"5,0",CHEF D’ŒUVRE ! Le film est absolument parfait ...,2
3,"5,0","Un film aussi novateur que complexe, dont la m...",3
4,"5,0",Christopher Nolan est sûrement l'un des seuls ...,4


In [23]:
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range(_nbPages):
        liste = getPage(_url + uri_pages + str(i + 1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale


In [24]:
liste_totale = getPages(nbPages, url)

In [25]:
liste_totale.head()

,Note,Description,key
0,"5,0",Après le chef d'oeuvre super-héroïque The Dark...,0
1,"5,0",C’est fou ce qu’on aime détester Christopher N...,1
2,"5,0",CHEF D’ŒUVRE ! Le film est absolument parfait ...,2
3,"5,0","Un film aussi novateur que complexe, dont la m...",3
4,"5,0",Christopher Nolan est sûrement l'un des seuls ...,4


In [26]:
liste_totale

,Note,Description,key
0,"5,0",Après le chef d'oeuvre super-héroïque The Dark...,0
1,"5,0",C’est fou ce qu’on aime détester Christopher N...,1
2,"5,0",CHEF D’ŒUVRE ! Le film est absolument parfait ...,2
3,"5,0","Un film aussi novateur que complexe, dont la m...",3
4,"5,0",Christopher Nolan est sûrement l'un des seuls ...,4
...,...,...,...
7209,"5,0",Ce film est un chef d'oeuvre. Christopher Nola...,1
7210,"5,0","Je j'ai vu en 2010, et 12 ans après c'est touj...",0
7211,"5,0",Ce film est un chef d'oeuvre. Christopher Nola...,1
7212,"5,0","Je j'ai vu en 2010, et 12 ans après c'est touj...",0


In [27]:
liste_totale.to_csv("allocine_inception_avis9.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)